#Extraction des Features

---



In [ ]:
def audio_to_lstm_features(file_path,
                          target_sr=16000,
                          max_duration=4.0,
                          n_mfcc=13,
                          hop_length=160,
                          n_fft=1024,
                          trim_silence=True,
                          top_db=25):
    """
    Convertit un fichier audio en caractéristiques MFCC pour un modèle LSTM.
    Retourne un tableau de forme (time_steps, n_features) pour une entrée LSTM.

    Args:
        file_path: Chemin vers le fichier audio
        target_sr: Taux d'échantillonnage cible
        max_duration: Durée maximale en secondes
        n_mfcc: Nombre de coefficients MFCC
        hop_length: Nombre d'échantillons entre les trames successives
        n_fft: Taille de la fenêtre FFT
        trim_silence: Si True, supprime les silences
        top_db: Seuil pour la suppression du silence

    Returns:
        features: Tableau numpy de forme (time_steps, n_mfcc)
    """

    # Chargement et prétraitement audio
    audio, sr = librosa.load(file_path, sr=target_sr)
    audio = librosa.util.normalize(audio)

    if trim_silence:
        audio = librosa.effects.trim(audio, top_db=top_db)[0]

    # Découpage/Padding pour longueur fixe
    target_length = int(max_duration * target_sr)
    if len(audio) > target_length:
        audio = audio[:target_length]
    else:
        padding = target_length - len(audio)
        audio = np.pad(audio, (0, padding), mode='constant')

    # Extraction MFCC
    mfcc = librosa.feature.mfcc(y=audio, sr=sr,
                               n_mfcc=n_mfcc,
                               hop_length=hop_length,
                               n_fft=n_fft,
                               fmin=50,
                               fmax=8000)

    # Normalisation par trame (le long des features)
    mfcc = (mfcc - np.mean(mfcc, axis=0)) / (np.std(mfcc, axis=0) + 1e-8)

    # Transposition pour avoir la forme (time_steps, n_features)
    mfcc = mfcc.T  # Shape: (time_steps, n_mfcc)

    return mfcc

In [ ]:
from tqdm.auto import tqdm

def process_files(df):
    results = []
    for _, row in tqdm(df.iterrows(), total=len(df)):
        try:
            features = audio_to_lstm_features(row['path']  , trim_silence=True)
            results.append({
                'path': row['path'],
                'features': features
            })
        except Exception as e:
            print(f"Erreur {row['path']}: {str(e)}")
    return results

# Exécution
features_data = process_files(df)

#Séparation des données

In [ ]:
X = np.array([x['features'] for x in features_data])  # Features audio
y = df['speaker_id'].values

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

callbacks = [
    ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=10, verbose=1),
    EarlyStopping(monitor='val_loss', patience=20, restore_best_weights=True, verbose=1),
    ModelCheckpoint(
        filepath='best_model.keras',
        save_best_only=True,
        monitor='val_accuracy',
        mode='max',
        verbose=1
    )
]

In [ ]:
# Encode labels correctement
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)  # Utilisation correcte de LabelEncoder

# Séparation des données en ensembles d'entraînement, validation et test
X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

# Affichage des dimensions des ensembles
print("Training Data Shape:", X_train.shape)
print("Validation Data Shape:", X_val.shape)
print("Test Data Shape:", X_test.shape)

Lstm : une seule couche
---



In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

model_lstm = Sequential([
    LSTM(128, return_sequences=False, input_shape=(X_train.shape[1], X_train.shape[2])),  # input shape = (T, 27)
    Dense(128, activation='relu'),
    Dropout(0.2),
    Dense(64, activation='relu'),
    Dense(y_categorical.shape[1], activation='softmax')
])

model_lstm.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model_lstm.summary()

In [ ]:
history4 = model_lstm.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=300,
    batch_size=128,
    callbacks=callbacks
)

In [ ]:
# Evaluate model
test_loss4, test_accuracy4 = model_lstm.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy4 * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# Plot Accuracy Curve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd plot
plt.plot(history4.history['accuracy'], label='Training Accuracy')
plt.plot(history4.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')

In [ ]:
# Plot Loss Curve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st plot
plt.plot(history4.history['loss'], label='Training Loss')
plt.plot(history4.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')

# Lstm : 2 couches

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

from tensorflow.keras.layers import BatchNormalization

model_lstm2_4 = Sequential([
    LSTM(128, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2])),
    BatchNormalization(),
    Dense(128, activation='relu'),
    Dropout(0.2),
    LSTM(64),
    BatchNormalization(),
    Dense(64, activation='relu'),
    Dense(y_categorical.shape[1], activation='softmax')
])

model_lstm2_4.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])


In [ ]:
history2_4 = model_lstm2_4.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=100,
    batch_size=128,
    callbacks=callbacks
)

In [ ]:
# Evaluate model
test_loss4, test_accuracy4 = model_lstm2_4.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracy4 * 100:.2f}%")

In [ ]:
# Plot Accuracy Curve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd plot
plt.plot(history2_4.history['accuracy'], label='Training Accuracy')
plt.plot(history2_4.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')

In [ ]:
# Plot Loss Curve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st plot
plt.plot(history2_4.history['loss'], label='Training Loss')
plt.plot(history2_4.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')

In [ ]:
import numpy as np
from sklearn.metrics import classification_report

y_pred_lstm = model_lstm2_4.predict(X_test)
y_pred_classes_lstm = np.argmax(y_pred_lstm , axis=1)
y_true_lstm = np.argmax(y_test, axis=1)

# Rapport de classification
target_names = [f"Classe {i}" for i in range(251)]
print(classification_report(y_true_lstm, y_pred_classes_lstm , target_names=target_names))
from sklearn.metrics import precision_score
precision_score(y_true_lstm , y_pred_classes_lstm , average='macro')

#Bi-Lstm

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Bidirectional, LSTM, Dense, Dropout, BatchNormalization

model = Sequential([
    Bidirectional(
        LSTM(128, return_sequences=True),   input_shape=(401, 13)  ),

    BatchNormalization(),
    Dropout(0.5),

    Bidirectional(LSTM(64)),
    BatchNormalization(),
    Dropout(0.5),

    Dense(256, activation='relu'),
    Dense(251, activation='softmax')
])

In [ ]:
modelbi3.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [ ]:
modelbi3.summary()

In [ ]:
historybic3 = modelbi3.fit(
    X_train, y_train,
    validation_data=(X_val, y_val),
    epochs=200,
    batch_size=128,
    callbacks=callbacks
)

In [ ]:
# Evaluate model
test_lossbi3, test_accuracybi3 = modelbi3.evaluate(X_test, y_test)
print(f"Test Accuracy: {test_accuracybi3 * 100:.2f}%")

In [ ]:
import matplotlib.pyplot as plt
# Plot Loss Curve
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 1)  # 1 row, 2 columns, 1st plot
plt.plot(historybic3.history['loss'], label='Training Loss')
plt.plot(historybic3.history['val_loss'], label='Validation Loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Curve')

In [ ]:
plt.figure(figsize=(12, 5))
plt.subplot(1, 2, 2)  # 1 row, 2 columns, 2nd plot
plt.plot(historybic3.history['accuracy'], label='Training Accuracy')
plt.plot(historybic3.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Curve')

In [ ]:
from sklearn.metrics import precision_score
precision_score(y_true, y_pred_classes, average='macro')

In [ ]:
import numpy as np
from sklearn.metrics import classification_report


y_pred = modelbi3.predict(X_test)
y_pred_classes = np.argmax(y_pred, axis=1)
y_true = np.argmax(y_test, axis=1)

# Rapport de classification
target_names = [f"Classe {i}" for i in range(251)]
print(classification_report(y_true, y_pred_classes, target_names=target_names))

In [ ]:
# Visualisation des performances
plt.plot(history2_4.history['val_accuracy'], label='LSTM2')
plt.plot(historybic3.history['val_accuracy'], label='BiLSTM')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()